In [1]:
from surgical_robotics_challenge import ecm_arm, camera
from ambf_client import Client
from autonomy_utils import ambf_utils
import rospy
import jupyros
from pathlib import Path
import cv2 as cv
import numpy as np 
rospy.init_node("img_node")

In [2]:
ambf_client = Client("juanclient")
ambf_client.connect()

camera_left = camera.Camera(ambf_client, "cameraL")
camera_right = camera.Camera(ambf_client, "cameraR")
ambf_cam_frame = camera.Camera(ambf_client, "CameraFrame")

ecm = ecm_arm.ECM(ambf_client, "CameraFrame")

print(f"Measured cp \n{ecm.measured_cp()}")
print(f"Measured jp \n{ecm.measured_jp()}")

Found Common Object Namespace as:  /ambf/env/
Measured cp 
[[          -1, 6.85663e-06,-3.67711e-06;
 -7.78039e-06,   -0.881271,    0.472612;
 -1.98367e-16,    0.472612,    0.881271]
[-8.55245e-07,    0.893574,     1.54166]]
Measured jp 
[0. 0. 0. 0.]


In [ ]:
ecm.servo_jp([-1000.5, 10000.0, -0.0, 0.0])
print(f"Measured jp \n{ecm.measured_jp()}")

# Thresholding


In [3]:

path = Path("./").resolve().parent / "examples/img/left_frame.jpeg"
print(path)
frame = cv.imread(str(path))
frame = cv.resize(frame, (640, 480), interpolation=cv.INTER_AREA)



/home/juan1995/research_juan/surgical_robotics_challenge/juan-scripts/examples/img/left_frame.jpeg


In [4]:
# Convert white pixels into black pixels
gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray, 220, 255, cv.THRESH_BINARY)
frame[thresh == 255] = 0

#Threshold the image
frame_HSV = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
# frame_threshold = cv.inRange(frame_HSV, (0,84,72), (180,254,197))
frame_threshold = cv.inRange(frame_HSV, (0,0,27), (180,84,255))
# Dilatation et erosion
kernel = np.ones((15,15), np.uint8)
img_dilation = cv.dilate(frame_threshold, kernel, iterations=1)
img_erode = cv.erode(img_dilation,kernel, iterations=1)
# clean all noise after dilatation and erosion
img_erode = cv.medianBlur(img_erode, 7)
#cv.imshow("frame",frame)
cv.imshow("test", img_erode)
cv.waitKey(0)
cv.destroyAllWindows()

# Segment AMBF

In [ ]:
from autonomy_utils.ambf_utils import ImageSaver
saver = ImageSaver()
selector = "left"


In [ ]:

frame = saver.get_current_frame(selector)

# path = Path("./").resolve().parent / "examples/output/needle_segmentation_pts0.jpeg"
# frame = cv.imread(str(path))
cv.imshow("test", frame)
cv.waitKey(0)
cv.destroyAllWindows()

# SIFT


In [47]:
import cv2
from autonomy_utils import pysift

keypoints, descriptors = pysift.computeKeypointsAndDescriptors(img_erode)


[<KeyPoint 0x7f60edcb1ba0>, <KeyPoint 0x7f60edcb1ab0>, <KeyPoint 0x7f60edcb1900>]


In [ ]:
print(keypoints)

In [45]:
img_1 = cv2.drawKeypoints(img_erode,keypoints,img_erode)

cv2.imshow("im",img_1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import cv2
# create SIFT feature extractor
sift = cv2.SIFT_create(nfeatures=18)

# detect features from the image
keypoints, descriptors = sift.detectAndCompute(img_erode, None)

# convert to point class
points = cv2.KeyPoint_convert(keypoints)
points = np.array(points)


In [6]:
print(points)
# Display image
final = np.copy(img_erode)
final = cv2.cvtColor(final, cv2.COLOR_GRAY2BGR)
for i in range(points.shape[0]):
    final = cv2.circle(final, (int(points[i, 0]), int(points[i, 1])), 3, (255, 0, 0), -1)

cv2.imshow("final",final)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[508.86438 417.9578 ]
 [516.01105 410.94846]
 [527.7544  327.28403]
 [527.7544  327.28403]]
